In [5]:
import keras
from keras.applications import vgg16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
from google.colab import drive
from sklearn.cluster import KMeans
import numpy as np

paths = []

drive.mount('/content/gdrive/', force_remount=False)

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [6]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
      img = tf.keras.utils.load_img(os.path.join(folder,filename), target_size=(224, 224))
      if img is not None:
        paths.append(os.path.join(folder,filename))
        images.append(img)
    return images

In [7]:
## VGG16 Model
base_model = vgg16.VGG16(weights='imagenet', include_top=False)
base_model.trainable =  False

inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
output = keras.layers.GlobalAveragePooling2D()(x)
model = keras.Model(inputs, output)

In [8]:
def get_feature_vector(image):
  x = tf.keras.utils.img_to_array(image)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  return model.predict(x)

def images2features(images):
  features = []
  for image in images:
    features.append(get_feature_vector(image)[0])
  return features

In [9]:
def train(features):
  kmeans_model = KMeans(n_clusters=2, random_state=0).fit(features)
  return kmeans_model

In [ ]:
# Read images for Kmeans
images1 = load_images_from_folder("/content/gdrive/MyDrive/Colab Notebooks/Dataset/Clouds and Lines/ba_202201/")
images2 = load_images_from_folder("/content/gdrive/MyDrive/Colab Notebooks/Dataset/Clouds and Lines/pm_20220125/")
images3 = load_images_from_folder("/content/gdrive/MyDrive/Colab Notebooks/Dataset/Clouds and Lines/pm_20220226/")
images4 = load_images_from_folder("/content/gdrive/MyDrive/Colab Notebooks/Dataset/Clouds and Lines/va_20220213/")

# Combine all lists
images = images1 + images2 + images3 + images4

# Extract features from the images
features = images2features(images)
# Train Kmeans model
kmeans_model = train(features)

In [11]:
# Predict output of same images using trained model
output_clusters = kmeans_model.predict(features)

In [16]:
import shutil

i = 0
for output in output_clusters:
  if output == 0:
    # save in cluster 0
    shutil.copy(paths[i], "/content/gdrive/MyDrive/Colab Notebooks/Dataset/Output/cluster0/")
  else:
    # save in cluster 1
    shutil.copy(paths[i], "/content/gdrive/MyDrive/Colab Notebooks/Dataset/Output/cluster1/")
  i = i+1

In [17]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(kmeans_model, f)

In [64]:
# Predict Function

# Load pickel model
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

def predict(image):
  feature_vec = get_feature_vector(image)[0]
  result = kmeans_model.predict([feature_vec])
  return result